In [4]:
if abspath(PROGRAM_FILE) == abspath(@__FILE__)
    # This condition is true if the script is run directly
    # (less common in notebooks, but good practice for standalone scripts)
    project_root = dirname(Pkg.project().path)
    cd(project_root)
elseif !isnothing(PROGRAM_FILE) && occursin("jupyter", lowercase(PROGRAM_FILE))
    # This condition is likely true when running in a Jupyter environment
    notebook_dir = dirname(@__FILE__)
    cd(notebook_dir)
else
    @info "Not running as a standalone script or in Jupyter, current working directory might be different."
end

println("Current working directory: ", pwd())

using Pkg
Pkg.activate("..")

In [ ]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays, Distributions
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using Plots
;

In [7]:
T = 10
c = 4.0
m⁰ = -0.6

β = 1.0
h = 0.0

popsize = 10^2
svd_trunc = TruncBond(15)
K = 20;

In [10]:
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
ψ_neutral = [ones(2,2) for t in 0:T];

In [34]:
prob_degree = Poisson(c)
prob_J = Uniform(0.0, 1.0)
prob_h = Dirac(0.0)

μ_pop = map(1:popsize) do p
    μ = rand_mpem2(ComplexF64, 2, 2, T)
    normalize!(μ)
    μ
end
bs = Vector{Vector{Float64}}[]
bs2var =  Matrix{Matrix{Float64}}[];

In [41]:
iterate_fourier_popdyn!(μ_pop, popsize, bs, bs2var, prob_degree, prob_J, prob_h, K, β, ϕᵢ, T; maxiter=1000, svd_trunc, tol=1e-10, σ=1/30)